## Prepare joined datasets for meteorology challenge

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from datetime import datetime

import pickle
import os

In [2]:
# working_folder = r'..\kaggle_data'
# os.chdir(working_folder)  # change to your own folder
os.chdir(r'C:\Users\User\Documents\DMBI_hackathon_2018\the_weather_channel-master\the_weather_channel-master\kaggle_data')

In [3]:
# Read data

# train sets
Tmax_train = pd.read_csv(r'train_maxTemp.csv')
Tmin_train = pd.read_csv(r'train_minTemp.csv')
Wind_train = pd.read_csv(r'train_Wind.csv')
Humidity_train = pd.read_csv(r'train_Humidity.csv')

# test sets
Tmax_test = pd.read_csv(r'test_maxTemp.csv')
Tmin_test = pd.read_csv(r'test_minTemp.csv')
Wind_test = pd.read_csv(r'test_Wind.csv')
Humidity_test = pd.read_csv(r'test_Humidity.csv')

# obs values
train_Tmax_obs = pd.read_csv(r'train_label.csv')
test_Tmax_obs = pd.read_csv(r'test_label.csv')

In [4]:
# join data sets by keys: Form, Validity date, City
# joined_train = pd.merge(Tmax_train, Tmin_train.drop(columns=['Basis date']), on=['Time', 'Validity date', 'City'], suffixes=('_tmax', '_tmin'))

joined_train_1 = pd.merge(Tmax_train, Tmin_train, on=['Time', 'Validity date', 'City'], suffixes=('_tmax', '_tmin'))
joined_train_2 = pd.merge(joined_train_1, Wind_train, on=['Time', 'Validity date', 'City'], suffixes=('', '_wind'))
joined_train = pd.merge(joined_train_2, Humidity_train, on=['Time', 'Validity date', 'City'], suffixes=('', '_humidity'))
joined_train.columns = ['Time', 'Basis date_tmax', 'Validity date', 'City',
       'Persist. value_tmax', 'EC_tmax', 'CO_tmax', 'C3_tmax', 'OH_tmax',
       'Basis date_tmin', 'Persist. value_tmin', 'EC_tmin', 'CO_tmin',
       'C3_tmin', 'OH_tmin', 'Basis date_wind', 'Persist. value_wind', 'EC_wind', 'CO_wind', 'C3_wind',
       'OH_wind', 'Basis date_humidity', 'Persist. value_humidity', 'EC_humidity',
       'C3_humidity']

joined_test = pd.merge(Tmax_test, Tmin_test, on=['Time', 'Validity date', 'City'], suffixes=('_tmax', '_tmin'))
joined_test = pd.merge(joined_test, Wind_test, on=['Time', 'Validity date', 'City'], suffixes=('', '_wind'))
joined_test = pd.merge(joined_test, Humidity_test, on=['Time', 'Validity date', 'City'], suffixes=('', '_humidity'))
joined_test.columns = ['Time', 'Basis date_tmax', 'Validity date', 'City',
       'Persist. value_tmax', 'EC_tmax', 'CO_tmax', 'C3_tmax', 'OH_tmax',
       'Basis date_tmin', 'Persist. value_tmin', 'EC_tmin', 'CO_tmin',
       'C3_tmin', 'OH_tmin', 'Basis date_wind', 'Persist. value_wind', 'EC_wind', 'CO_wind', 'C3_wind',
       'OH_wind', 'Basis date_humidity', 'Persist. value_humidity', 'EC_humidity',
       'C3_humidity']

In [5]:
# Casting string values to float (the model can fit only float values)

# casting 'Validity date' to datetime.timestamp
def castDates(df):
    df['Validity date'] = df['Validity date'].apply(lambda x: (datetime.strptime(x, '%d-%m-%y')).timestamp())

def merge12(df):
    df1 = df[df['Time']== 1]
    df2 = df[df['Time']== 2]
    merged_df = pd.merge(df1, df2, on=['Validity date', 'City'], suffixes=('_1', '_2'))
    merged_df.drop(['Time_1', 'Time_2'], axis=1, inplace=True)
    return merged_df

def drop_basis(df):
    return df.drop([x for x in df.columns if x.startswith('Basis date')], axis=1)

def prepare_and_process_set(df):
    df['City'] = leCity.transform(df['City'])
    castDates(df)
    joined_12 = merge12(df)
    joined_12 = drop_basis(joined_12)
    return joined_12
    
leCity = preprocessing.LabelEncoder()
leCity.fit(joined_train['City'])
joined_train = prepare_and_process_set(joined_train)
joined_test = prepare_and_process_set(joined_test)


In [6]:
# Adding city data from the IMS' site

stationsData = pd.read_csv(r'stations_dist.csv')
joined_train = joined_train.merge(stationsData, left_on='City', right_on='cityID',how='outer')
joined_test = joined_test.merge(stationsData, left_on='City', right_on='cityID',how='outer')

joined_train.head()

,Validity date,City,Persist. value_tmax_1,EC_tmax_1,CO_tmax_1,C3_tmax_1,OH_tmax_1,Persist. value_tmin_1,EC_tmin_1,CO_tmin_1,...,C3_humidity_2,cityID,stationID,name,E,N,elevation,elevationWindMet,CoastalDist,archiveID
0,1.451686e+09,0,12,11.0,11.0,12.0,13.0,6.0,7.0,6.0,...,35,0,16,Afula,35.277,32.596,60,70,33.34,5811
1,1.451772e+09,0,11,11.0,11.0,14.0,13.0,7.0,9.0,8.0,...,51,0,16,Afula,35.277,32.596,60,70,33.34,5811
2,1.451858e+09,0,15,14.0,16.0,14.0,17.0,6.0,8.0,11.0,...,58,0,16,Afula,35.277,32.596,60,70,33.34,5811
3,1.451945e+09,0,16,19.0,17.0,17.0,20.0,10.0,9.0,10.0,...,49,0,16,Afula,35.277,32.596,60,70,33.34,5811
4,1.452031e+09,0,20,20.0,18.0,18.0,20.0,6.0,8.0,12.0,...,45,0,16,Afula,35.277,32.596,60,70,33.34,5811


In [7]:
# Generate seasonal features

# joined_train = pd.read_pickle(r'joined_train_12_daily_features_df.pickle')
# joined_test  = pd.read_pickle(r'joined_test_12_daily_features_df.pickle')
joined_train = joined_train.sort_values(['Validity date','cityID'])
joined_train['season'] = 0
startDayOfSeason = 0 
endDayOfSeason = 59*15 
joined_train.season.iloc[startDayOfSeason:endDayOfSeason+1]='Winter'
startDayOfSeason, endDayOfSeason = endDayOfSeason, endDayOfSeason+92*15
joined_train.season.iloc[startDayOfSeason+1:endDayOfSeason+1]='Spring'
startDayOfSeason, endDayOfSeason = endDayOfSeason, endDayOfSeason+92*15
joined_train.season.iloc[startDayOfSeason+1:endDayOfSeason+1]='Summer'
startDayOfSeason, endDayOfSeason = endDayOfSeason, endDayOfSeason+91*15
joined_train.season.iloc[startDayOfSeason+1:endDayOfSeason+1]='Autumn'
startDayOfSeason, endDayOfSeason = endDayOfSeason, endDayOfSeason+90*15
joined_train.season.iloc[startDayOfSeason+1:endDayOfSeason+1]='Winter'
startDayOfSeason, endDayOfSeason = endDayOfSeason, endDayOfSeason+92*15
joined_train.season.iloc[startDayOfSeason+1:endDayOfSeason+1]='Spring'
startDayOfSeason, endDayOfSeason = endDayOfSeason, endDayOfSeason+92*15
joined_train.season.iloc[startDayOfSeason+1:endDayOfSeason+1]='Summer'
startDayOfSeason, endDayOfSeason = endDayOfSeason, endDayOfSeason+91*15
joined_train.season.iloc[startDayOfSeason+1:endDayOfSeason+1]='Autumn'
startDayOfSeason, endDayOfSeason = endDayOfSeason, endDayOfSeason+30*15
joined_train.season.iloc[startDayOfSeason+1:endDayOfSeason+1]='Winter'

joined_test = joined_test.sort_values(['Validity date','cityID'])
joined_test['season'] = 0
startDayOfSeason = 0 
endDayOfSeason = 59*15 
joined_test.season.iloc[startDayOfSeason:endDayOfSeason+1]='Winter'
startDayOfSeason, endDayOfSeason = endDayOfSeason, endDayOfSeason+31*15
joined_test.season.iloc[startDayOfSeason+1:endDayOfSeason+1]='Spring'


C:\Users\User\Anaconda3_6\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
# Generating per day features
# joined_train = pd.read_pickle(r'joined_train_with_metro_data.pickle')
# joined_test = pd.read_pickle(r'joined_test_with_metro_data.pickle')

def genModelFeatAvg():
    for df in [joined_train,joined_test]:
        for feat in ['tmax','tmin','wind','humidity']:
#             Avg for each persist feat when 1 and 2 are combined
            df['persist_'+feat+'_daily_avg']=sum([(df['Persist. value_'+feat+'_'+i]) for i in ["1","2"]])/2
#             Avg for each feat when 1 and 2 are combined
            if feat=='humidity':
                df['models_'+feat+'_daily_avg']=sum([(df['EC_'+feat+'_'+i]+df['C3_'+feat+'_'+i]) for i in ["1","2"]])/4
            else:
                df['models_'+feat+'_daily_avg']=sum([(df['EC_'+feat+'_'+i]+df['CO_'+feat+'_'+i]+df['C3_'+feat+'_'+i]+df['OH_'+feat+'_'+i]) for i in ["1","2"]])/8
#             Avg for each feat when 1 and 2 are apart
            for i in ['1','2']:
                if feat=='humidity':
                    df['models_'+feat+'_'+i+'_avg']=(df['EC_'+feat+'_'+i]+df['C3_'+feat+'_'+i])/2
                else:
                    df['models_'+feat+'_'+i+'_avg']=(df['EC_'+feat+'_'+i]+df['CO_'+feat+'_'+i]+df['C3_'+feat+'_'+i]+df['OH_'+feat+'_'+i])/4

def genTempRange():
    for df in [joined_train,joined_test]:
        for model in ['EC','CO','C3','OH']:
            for i in ['1','2']:
                df[model+'_range_temp_'+i]=df[model+'_tmax_'+i]-df[model+'_tmin_'+i]
                
def genDayTempAvg():
    for df in [joined_train,joined_test]:
        for model in ['EC','CO','C3','OH']:
            df[model+'_daily_avg_temp']=sum([df[model+'_tmax_'+i]+df[model+'_tmin_'+i] for i in ["1","2"]])/4
def genDistPersistModelsAvg():
    for df in [joined_train,joined_test]:
        for feat in ['tmax','tmin','wind','humidity']:
            df['dist_persist_models_avg_'+feat+'_daily']=abs(df['models_'+feat+'_daily_avg']-df['persist_'+feat+'_daily_avg'])
            for i in ['1','2']:
                df['dist_persist_models_avg_'+feat+'_'+i]=abs(df['models_'+feat+'_'+i+'_avg']-df['Persist. value_'+feat+'_'+i])
def reliableModelsAvg():
    for df in [joined_train,joined_test]:
        for feat in ['tmax','tmin','wind']:
#             Avg for each feat when 1 and 2 are combined
            df['reliable_models_'+feat+'_daily_avg']=sum([(df['EC_'+feat+'_'+i]+df['CO_'+feat+'_'+i]+df['C3_'+feat+'_'+i]) for i in ["1","2"]])/6
#             Avg for each feat when 1 and 2 are apart
            for i in ['1','2']:
                df['reliable_models_'+feat+'_'+i+'_avg']=(df['EC_'+feat+'_'+i]+df['CO_'+feat+'_'+i]+df['C3_'+feat+'_'+i])/3
joined_train = pd.get_dummies(joined_train, columns=['City'])
joined_test = pd.get_dummies(joined_test, columns=['City'])

joined_train = pd.get_dummies(joined_train, columns=['season'])
joined_test = pd.get_dummies(joined_test, columns=['season'])

genModelFeatAvg()
genTempRange()
genDayTempAvg()
genDistPersistModelsAvg()
reliableModelsAvg()
print(list(joined_test.columns.values))
joined_train.head()
# joined_train.to_pickle('joined_train_12_daily_features_df.pickle')
# joined_test.to_pickle('joined_test_12_daily_features_df.pickle')

['Validity date', 'Persist. value_tmax_1', 'EC_tmax_1', 'CO_tmax_1', 'C3_tmax_1', 'OH_tmax_1', 'Persist. value_tmin_1', 'EC_tmin_1', 'CO_tmin_1', 'C3_tmin_1', 'OH_tmin_1', 'Persist. value_wind_1', 'EC_wind_1', 'CO_wind_1', 'C3_wind_1', 'OH_wind_1', 'Persist. value_humidity_1', 'EC_humidity_1', 'C3_humidity_1', 'Persist. value_tmax_2', 'EC_tmax_2', 'CO_tmax_2', 'C3_tmax_2', 'OH_tmax_2', 'Persist. value_tmin_2', 'EC_tmin_2', 'CO_tmin_2', 'C3_tmin_2', 'OH_tmin_2', 'Persist. value_wind_2', 'EC_wind_2', 'CO_wind_2', 'C3_wind_2', 'OH_wind_2', 'Persist. value_humidity_2', 'EC_humidity_2', 'C3_humidity_2', 'cityID', 'stationID', 'name', 'E', 'N', 'elevation ', 'elevationWindMet', 'CoastalDist', 'archiveID', 'City_0', 'City_1', 'City_2', 'City_3', 'City_4', 'City_5', 'City_6', 'City_7', 'City_8', 'City_9', 'City_10', 'City_11', 'City_12', 'City_13', 'City_14', 'season_Spring', 'season_Winter', 'persist_tmax_daily_avg', 'models_tmax_daily_avg', 'models_tmax_1_avg', 'models_tmax_2_avg', 'persist_

,Validity date,Persist. value_tmax_1,EC_tmax_1,CO_tmax_1,C3_tmax_1,OH_tmax_1,Persist. value_tmin_1,EC_tmin_1,CO_tmin_1,C3_tmin_1,...,dist_persist_models_avg_humidity_2,reliable_models_tmax_daily_avg,reliable_models_tmax_1_avg,reliable_models_tmax_2_avg,reliable_models_tmin_daily_avg,reliable_models_tmin_1_avg,reliable_models_tmin_2_avg,reliable_models_wind_daily_avg,reliable_models_wind_1_avg,reliable_models_wind_2_avg
0,1.451686e+09,12,11.0,11.0,12.0,13.0,6.0,7.0,6.0,6.0,...,29.5,11.666667,11.333333,12.000000,6.500000,6.333333,6.666667,7.500000,6.666667,8.333333
729,1.451686e+09,15,14.0,12.0,14.0,14.0,10.0,12.0,9.0,10.0,...,29.5,13.333333,13.333333,13.333333,10.833333,10.333333,11.333333,12.166667,13.666667,10.666667
1458,1.451686e+09,10,10.0,11.0,12.0,9.0,5.0,7.0,7.0,7.0,...,28.0,10.833333,11.000000,10.666667,6.833333,7.000000,6.666667,10.500000,9.333333,11.666667
2187,1.451686e+09,15,11.0,14.0,14.0,13.0,8.0,8.0,7.0,8.0,...,32.0,13.166667,13.000000,13.333333,7.666667,7.666667,7.666667,4.666667,5.000000,4.333333
2916,1.451686e+09,19,12.0,14.0,15.0,13.0,9.0,4.0,7.0,6.0,...,7.0,13.666667,13.666667,13.666667,5.833333,5.666667,6.000000,6.833333,6.666667,7.000000


### Create timeseries

In [9]:
def merge_n_days(df, n):
    df_list = []  # init
    for city in df['name'].unique():
        # subset to city
        local_df = df[df['name'] == city]
        local_df.drop(['cityID', 'name', 'stationID', 'archiveID'], axis=1, inplace=True)
        sorted_local_df = local_df.sort_values(['Validity date'], ascending=False)
        shifted_df_list = [sorted_local_df.shift(-1*i) for i in range(1, n+1)]  # for each day
        
        for i in range(n):  # for each day
            shifted_df = shifted_df_list[i]
            back = -1*(i+1)
            sorted_local_df = sorted_local_df.join(shifted_df.rename(columns=lambda x: x+f"_{back}"))
            sorted_local_df.drop([f'Validity date_{back}'], axis=1, inplace=True)
            sorted_local_df.drop([f'City_{i}_{back}' for i in range(15)], axis=1, inplace=True)
            sorted_local_df.drop([f'CoastalDist_{back}', f'elevation _{back}', f'N_{back}', f'E_{back}', ], axis=1, inplace=True)
            
        df_list.append(sorted_local_df)
    return pd.concat(df_list)  # for all cities


def create_list_of_1_to_n_timeseries(train_df, test_df, n):
    ts_df_train_list = [merge_n_days(train_df, i) for i in range(1, n)]
    ts_df_test_list = [merge_n_days(test_df, i) for i in range(1, n)]
    return ts_df_train_list, ts_df_test_list

# change here to your train + test
ts_df_train_list, ts_df_test_list = create_list_of_1_to_n_timeseries(joined_train, joined_test, 10)

C:\Users\User\Anaconda3_6\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:

X_train = joined_train.drop(['cityID', 'name', 'stationID', 'archiveID'], axis=1)
X_test = joined_test.drop(['cityID', 'name', 'stationID', 'archiveID'], axis=1)
X_test['season_Autumn'] = 0
X_test['season_Summer'] = 0

### Train Model

In [27]:
# Create a linear Regression model
reg = LinearRegression()
# reg.fit(joined_train, train_Tmax_obs['observedMaxTemp'])
reg.fit(X_train, train_Tmax_obs['observedMaxTemp'])  # only for 1

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [28]:
predicted_values = reg.predict(X_test)

In [29]:
# Accuracy of the model

# Calculate rmse by the difference square of the model predicted values and the actual values (obs)
rmse = np.sqrt(metrics.mean_squared_error(test_Tmax_obs['observedMaxTemp'], predicted_values))

print(rmse)

# old: 1.75672391465
# old: 1.65650385966

3.06829588506


In [31]:
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, accuracy_score, average_precision_score, adjusted_rand_score, adjusted_mutual_info_score, confusion_matrix
# polynomial

poly = PolynomialFeatures(2)
X_train_poly = poly.fit_transform(X_train)
# X_train_poly_df = pd.DataFrame(X_train_poly)
# X_train_poly_df_1hot = pd.get_dummies(X_train_poly_df, columns=['City'])

X_test_poly = poly.fit_transform(X_test)
# X_test_poly_df = pd.DataFrame(X_test_poly)
# X_test_poly_df_1hot = pd.get_dummies(X_test_poly_df, columns=['City'])

In [32]:
reg = LinearRegression()
# reg.fit(joined_train, train_Tmax_obs['observedMaxTemp'])
reg.fit(X_train_poly, train_Tmax_obs['observedMaxTemp'])  # only for 1
predicted_values = reg.predict(X_test_poly)
rmse = np.sqrt(metrics.mean_squared_error(test_Tmax_obs['observedMaxTemp'], predicted_values))

print(rmse)

634.238504696


In [34]:
# scaled
reg = LinearRegression()
# reg.fit(joined_train, train_Tmax_obs['observedMaxTemp'])
reg.fit(scale(X_train), train_Tmax_obs['observedMaxTemp'])  # only for 1
predicted_values = reg.predict(scale(X_test))
rmse = np.sqrt(metrics.mean_squared_error(test_Tmax_obs['observedMaxTemp'], predicted_values))

print(rmse)

5866561428.07


C:\Users\User\Anaconda3_6\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [35]:
for col in X_train.columns:
    print(col)

Validity date
Persist. value_tmax_1
EC_tmax_1
CO_tmax_1
C3_tmax_1
OH_tmax_1
Persist. value_tmin_1
EC_tmin_1
CO_tmin_1
C3_tmin_1
OH_tmin_1
Persist. value_wind_1
EC_wind_1
CO_wind_1
C3_wind_1
OH_wind_1
Persist. value_humidity_1
EC_humidity_1
C3_humidity_1
Persist. value_tmax_2
EC_tmax_2
CO_tmax_2
C3_tmax_2
OH_tmax_2
Persist. value_tmin_2
EC_tmin_2
CO_tmin_2
C3_tmin_2
OH_tmin_2
Persist. value_wind_2
EC_wind_2
CO_wind_2
C3_wind_2
OH_wind_2
Persist. value_humidity_2
EC_humidity_2
C3_humidity_2
E
N
elevation 
elevationWindMet
CoastalDist
City_0
City_1
City_2
City_3
City_4
City_5
City_6
City_7
City_8
City_9
City_10
City_11
City_12
City_13
City_14
season_Autumn
season_Spring
season_Summer
season_Winter
persist_tmax_daily_avg
models_tmax_daily_avg
models_tmax_1_avg
models_tmax_2_avg
persist_tmin_daily_avg
models_tmin_daily_avg
models_tmin_1_avg
models_tmin_2_avg
persist_wind_daily_avg
models_wind_daily_avg
models_wind_1_avg
models_wind_2_avg
persist_humidity_daily_avg
models_humidity_daily_avg


In [ ]:
X_train.drop(['E', 'N', 'elevation', 'elevationWindMet', 'CoastalDist'])

In [30]:
X_test.columns

Index(['Validity date', 'Persist. value_tmax_1', 'EC_tmax_1', 'CO_tmax_1',
       'C3_tmax_1', 'OH_tmax_1', 'Persist. value_tmin_1', 'EC_tmin_1',
       'CO_tmin_1', 'C3_tmin_1',
       ...
       'reliable_models_tmax_1_avg', 'reliable_models_tmax_2_avg',
       'reliable_models_tmin_daily_avg', 'reliable_models_tmin_1_avg',
       'reliable_models_tmin_2_avg', 'reliable_models_wind_daily_avg',
       'reliable_models_wind_1_avg', 'reliable_models_wind_2_avg',
       'season_Autumn', 'season_Summer'],
      dtype='object', length=110)

### Save for submission

In [10]:
# Create a new data frame containing 2 columns:
# 1. key -concatenation of station name and date
# 2. predicted values

# dates = joined_test['Validity date'].apply(lambda x: datetime.fromtimestamp(x).strftime('%d-%m-%y') + '_')
# cities = leCity.inverse_transform(joined_test['City'])
# prediction_data = pd.DataFrame()
# prediction_data['validityDate_city'] = dates + cities
# prediction_data['predictedMaxTemp'] = predicted_values

# prediction_data.to_csv(r'sampleSubmission.csv', index=False)